# Libraries

In [1]:
import pandas as pd
from getpass import getuser
import os
import numpy as np

# Input Directories

In [2]:
# Get the current user's name
user = getuser()
directory_in = f"C:/Users/{user}/Documents/GitHub/davis-cup/data/davis matches/combined_davis.xlsx"
df = pd.read_excel(directory_in)

# create and rename variables

In [3]:
# rename team variable
df.rename(columns={'team_name': 'team', 'court pace rating': 'court_pace_rating'}, inplace=True)

# remove the "match status" column
df = df.drop(columns=["match status"], errors='ignore')

#  modify the "match" column to keep only the number
df['match'] = df['match'].str.extract('(\d+)', expand=False).astype(int)

# convert the "team" column values to lowercase
df['team'] = df['team'].str.lower()

# split the "date" column into "date_start" and "date_end"
df[['date_start', 'date_end']] = df['date'].str.split(' - ', expand=True)

# convert the dates to the desired format (dd/mm/yyyy)
df['date_start'] = pd.to_datetime(df['date_start'] + ' ' + df['date_end'].str[-4:]).dt.strftime('%d/%m/%Y')
df['date_end'] = pd.to_datetime(df['date_end']).dt.strftime('%d/%m/%Y')

# remove the original "date" column
df = df.drop(columns=['date'])

# split the "venue" column into "venue_name", "city", and "country"
venue_split = df['venue'].str.split(', ', expand=True)

# handling cases where split does not produce exactly 3 parts
df['venue_name'] = venue_split[0]
df['city'] = venue_split[1]
df['country'] = venue_split[2]

# remove the original "venue" column
df = df.drop(columns=['venue'])

# split the "surface" column into "main_surface", "specific_surface", and "indoor_outdoor"
surface_split = df['surface'].str.split(' - ', expand=True)

# further split the second part to separate specific surface and indoor/outdoor
surface_details = surface_split[1].str.split(', ', expand=True)

# assign the split components to new columns
df['main_surface'] = surface_split[0]
df['specific_surface'] = surface_details[0]
df['indoor_outdoor'] = surface_details[1]

# remove the original "surface" column
df = df.drop(columns=['surface'])

# Rename the columns
df.rename(columns={'player1': 'p1t1', 'player2': 'p2t1'}, inplace=True)

# Assign teams to specific players when the team value is missing
df['team'] = df.apply(lambda row: 'france' if row['p1t1'] in ['Lucas Pouille', 'Pierre-Hugues Herbert', 'Richard Gasquet'] 
                      else 'canada' if row['p1t1'] == 'Filip Peliwo' 
                      else 'usa' if row['p1t1'] == 'Franklin Tiafoe' 
                      else row['team'], axis=1)

print(df.shape)


(988, 32)


<>:8: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\d'
C:\Users\aldi\AppData\Local\Temp\ipykernel_21256\1926264600.py:8: SyntaxWarning: invalid escape sequence '\d'
  df['match'] = df['match'].str.extract('(\d+)', expand=False).astype(int)


# Check number of players in each team for each year 

In [4]:
# Group by 'team' and 'year', then count the number of unique 'player1'
unique_players_per_team_year = df.groupby(['team', 'year'])['p1t1'].nunique()

# Convert the Series to a DataFrame
result_table = unique_players_per_team_year.reset_index(name='unique_player_count')

# Pivot the DataFrame to create the desired table
pivot_table = result_table.pivot(index='team', columns='year', values='unique_player_count')

# Reindex the columns to include all years from 2014 to 2023
all_years = range(2014, 2024)
pivot_table = pivot_table.reindex(columns=all_years)

# Display the resulting table
pivot_table



year,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
team,,,,,,,,,,
argentina,3.0,4.0,7.0,2.0,NaN,3.0,NaN,NaN,NaN,NaN
australia,4.0,5.0,3.0,5.0,3.0,3.0,NaN,NaN,4.0,4.0
austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
belgium,2.0,4.0,3.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN
brazil,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
canada,2.0,5.0,3.0,3.0,3.0,3.0,NaN,NaN,3.0,3.0
croatia,NaN,4.0,5.0,4.0,4.0,NaN,NaN,3.0,3.0,NaN
czechia,4.0,4.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN,2.0
finland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0


In [5]:
# Highlight values greater than 5
def highlight(value):
    if pd.notnull(value) and value > 5:
        return 'background-color: yellow'
    return ''

styled_table = pivot_table.style.applymap(lambda x: highlight(x))

# Display the styled table
styled_table


C:\Users\aldi\AppData\Local\Temp\ipykernel_21256\1564758840.py:7: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_table = pivot_table.style.applymap(lambda x: highlight(x))


year,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
team,,,,,,,,,,
argentina,3.000000,4.000000,7.000000,2.000000,nan,3.000000,nan,nan,nan,nan
australia,4.000000,5.000000,3.000000,5.000000,3.000000,3.000000,nan,nan,4.000000,4.000000
austria,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
belgium,2.000000,4.000000,3.000000,5.000000,5.000000,nan,nan,nan,nan,nan
brazil,nan,3.000000,nan,nan,nan,nan,nan,nan,nan,nan
canada,2.000000,5.000000,3.000000,3.000000,3.000000,3.000000,nan,nan,3.000000,3.000000
croatia,nan,4.000000,5.000000,4.000000,4.000000,nan,nan,3.000000,3.000000,nan
czechia,4.000000,4.000000,3.000000,2.000000,nan,nan,nan,nan,nan,2.000000
finland,nan,nan,nan,nan,nan,nan,nan,nan,nan,4.000000


In [6]:
filtered_df = df[(df['year'] == 2018) & (df['team'] == 'france')]

unique_players_team = filtered_df['p1t1'].unique()

# Display the unique players for 2018 and France
print(unique_players_team)


['Adrian Mannarino' 'Richard Gasquet' 'Pierre-Hugues Herbert'
 'Lucas Pouille' 'Jeremy Chardy' 'Benoit Paire' 'Julien Benneteau'
 'Nicolas Mahut' 'Jo-Wilfried Tsonga']


# Apply correct format to variables

In [7]:
# # Example: List of columns for different data types
numeric_cols = ['match','set1', 'set2', 'set3', 'set4', 'set5', 'tb1', 'tb2', 'tb3', 'tb4', 'tb5',
                'single_ranking_player1', 'doubles_ranking_player1', 
                'single_ranking_player2', 'doubles_ranking_player2']
# date_cols = ['date_start', 'date_end', 'dob_player1', 'dob_player2']
# categorical_cols = ['team', 'main_surface', 'specific_surface', 'indoor_outdoor']
# string_cols = ['p1t1', 'p2t1', 'venue_name', 'city', 'country']

# Convert numeric columns to numeric types
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# # Convert date columns to datetime
# df[date_cols] = df[date_cols].apply(pd.to_datetime, errors='coerce')

# # Convert categorical columns to category dtype
# df[categorical_cols] = df[categorical_cols].astype('category')

# # Convert string columns to string dtype, ensuring missing values remain empty
# df[string_cols] = df[string_cols].applymap(lambda x: '' if pd.isna(x) else str(x))


# Check number of matches for each group under the old and new structure 

In [8]:
def check_team_counts(df):
    # Group by 'team', 'year', and 'stage' and count the occurrences
    grouped = df.groupby(['team', 'year', 'stage']).size().reset_index(name='count')

    # Define the conditions based on the year
    condition1 = (grouped['year'] < 2019) & (grouped['count'] <= 3)
    condition2 = (grouped['year'] >= 2019) & ((grouped['count'] < 2) | (grouped['count'] > 3))
    
    # Combine the conditions
    differing_counts = grouped[condition1 | condition2]

    return differing_counts

# Assuming df is your DataFrame, you can call the function like this:
differing_counts = check_team_counts(df)

# Display the results
print(differing_counts)


Empty DataFrame
Columns: [team, year, stage, count]
Index: []


# pair rows 

## pair function

In [9]:
def pair_rows(df):
    paired_rows = []
    unmatched_rows = []

    for i in range(0, len(df), 2):
        if i + 1 < len(df):
            row1 = df.iloc[i]
            row2 = df.iloc[i + 1]
            
            # Check conditions and collect reasons if unmatched
            reasons = []
            if row1['match'] != row2['match']:
                reasons.append(f"match mismatch: {row1['match']} vs {row2['match']}")
            if row1['year'] != row2['year']:
                reasons.append(f"year mismatch: {row1['year']} vs {row2['year']}")
            if row1['date_start'] != row2['date_start']:
                reasons.append(f"date_start mismatch: {row1['date_start']} vs {row2['date_start']}")
            if row1['date_end'] != row2['date_end']:
                reasons.append(f"date_end mismatch: {row1['date_end']} vs {row2['date_end']}")
            if row1['p1t1'] == row2['p1t1']:
                reasons.append(f"p1t1 same: {row1['p1t1']} vs {row2['p1t1']}")
            
            if row1['team'] == row2['team']:
                reasons.append(f"same team: {row1['team']} vs {row2['team']}")

            # If no reasons were collected, pair the rows
            if not reasons:
                paired_row = [
                    row1['ball'], row1['stage'], row1['match'], row1['year'],
                    row1['p1t1'], row1['p2t1'], row2['p1t1'], row2['p2t1'],
                    row1['team'], row2['team'],
                    row1['date_start'], row1['date_end'],
                    row1['venue_name'], row1['city'], row1['country'],
                    row1['main_surface'], row1['specific_surface'], row1['indoor_outdoor'],
                    row1['court_pace_rating']
                ]
                paired_rows.append(paired_row)
            else:
                # Collect the reason(s) for why these rows were not paired, including actual values
                unmatched_rows.append({
                    'row1_index': i,
                    'row2_index': i + 1,
                    'reasons': "; ".join(reasons)
                })

    # Create DataFrames from the paired and unmatched rows
    paired_df = pd.DataFrame(paired_rows, columns=[
        'ball', 'stage', 'match', 'year',
        'p1t1', 'p2t1', 'p1t2', 'p2t2', 
        'team1', 'team2',  
        'date_start', 'date_end', 
        'venue_name', 'city', 'country', 
        'main_surface', 'specific_surface', 'indoor_outdoor', 
        'court_pace_rating'
    ])
    
    unmatched_df = pd.DataFrame(unmatched_rows)
    
    return paired_df, unmatched_df


## implement function

In [10]:
# Pair the rows in the extracted dataframe
df_paired, df_unmatched = pair_rows(df)

# Replace specific player names to match with data from ATP matches
df_paired.replace({
    'p1t1': {
        'Jan-Lennard Struff': 'Jan Lennard Struff',
        'Juan Martin Del Potro': 'Juan Martin del Potro',
        'Albert Ramos-Vinolas': 'Albert Ramos',
        'Felix Auger-Aliassime': 'Felix Auger Aliassime',
        'Pierre-Hugues Herbert': 'Pierre Hugues Herbert',
        'Marc-Andrea Huesler': 'Marc Andrea Huesler',
        'Roman Khassanov': 'Roman Hassanov'
    },
    'p2t1': {
        'Jan-Lennard Struff': 'Jan Lennard Struff',
        'Juan Martin Del Potro': 'Juan Martin del Potro',
        'Albert Ramos-Vinolas': 'Albert Ramos',
        'Felix Auger-Aliassime': 'Felix Auger Aliassime',
        'Pierre-Hugues Herbert': 'Pierre Hugues Herbert',
        'Marc-Andrea Huesler': 'Marc Andrea Huesler',
        'Roman Khassanov': 'Roman Hassanov'
    },
    'p1t2': {
        'Jan-Lennard Struff': 'Jan Lennard Struff',
        'Juan Martin Del Potro': 'Juan Martin del Potro',
        'Albert Ramos-Vinolas': 'Albert Ramos',
        'Felix Auger-Aliassime': 'Felix Auger Aliassime',
        'Pierre-Hugues Herbert': 'Pierre Hugues Herbert',
        'Marc-Andrea Huesler': 'Marc Andrea Huesler',
        'Roman Khassanov': 'Roman Hassanov'
    },
    'p2t2': {
        'Jan-Lennard Struff': 'Jan Lennard Struff',
        'Juan Martin Del Potro': 'Juan Martin del Potro',
        'Albert Ramos-Vinolas': 'Albert Ramos',
        'Felix Auger-Aliassime': 'Felix Auger Aliassime',
        'Pierre-Hugues Herbert': 'Pierre Hugues Herbert',
        'Marc-Andrea Huesler': 'Marc Andrea Huesler',
        'Roman Khassanov': 'Roman Hassanov'
    }
}, inplace=True)

# Now df_paired has the replaced player names


In [11]:
# Print the number of observations for each DataFrame
print("Number of observations in df:", df.shape[0])
print("Number of observations in df_paired:", df_paired.shape[0])
print("Number of observations in df_unmatched:", df_unmatched.shape[0])


Number of observations in df: 988
Number of observations in df_paired: 494
Number of observations in df_unmatched: 0


# delete wrong rows

In [12]:
# Define the condition for rows to be deleted
condition = (
    (df_paired['year'] == 2019) & 
    (
        # Condition for russia vs spain
        ((df_paired['team1'] == 'russia') & (df_paired['team2'] == 'spain')) | 
        ((df_paired['team1'] == 'spain') & (df_paired['team2'] == 'russia')) |
        
        # Condition for russia vs great britain
        ((df_paired['team1'] == 'russia') & (df_paired['team2'] == 'great britain')) | 
        ((df_paired['team1'] == 'great britain') & (df_paired['team2'] == 'russia'))
    )
)

# Delete the rows that meet the condition
df_paired = df_paired[~condition]


# check observations for each group

In [13]:
# Group by date, year, and Venue, then count observations
grouped_counts = df_paired.groupby(['date_end', 'year', 'venue_name', 'stage','team1']).size()

# Define a function to check if counts meet the specified criteria
def check_count(year, count):
    if year <= 2018:
        return 3 <= count <= 5
    else:
        return count <= 3

# Apply the function to each row in the grouped data and create a new column 'Count'
correct_counts_df = grouped_counts.reset_index()
correct_counts_df['count'] = correct_counts_df.apply(lambda row: row[0], axis=1)

# Filter rows where counts do not meet the criteria
incorrect_counts_df = correct_counts_df[~correct_counts_df.apply(lambda row: check_count(row['year'], row['count']), axis=1)]

# Check if the DataFrame is empty, indicating all matches are in the correct range
if incorrect_counts_df.empty:
    print("All matches are in the correct range.")
else:
    # Display date, year, Venue, and Count when correct_counts is False
    print(incorrect_counts_df[['date_end', 'year','stage', 'venue_name','team1', 'count']])


      date_end  year                      stage        venue_name    team1  \
44  06/04/2014  2014  WORLD GROUP  QUARTERFINAL   Ariake Coliseum  czechia   

    count  
44      1  


# export dataframe

In [14]:
directory_out = f"C:/Users/{user}/Documents/GitHub/davis-cup/data/"

# Save the concatenated DataFrame to a new Excel file
df_paired.to_excel(os.path.join(directory_out, 'website_df.xlsx'), index=False)

# Verify the operation
print("All files have been successfully combined and saved to 'website_df.xlsx'.")

All files have been successfully combined and saved to 'website_df.xlsx'.
